## Create avi video from frames

### Input dir: dir with sequential frames

In [16]:
import cv2
import os
import re

# ----------------- CONFIGURATION -----------------
# input_dir = "/home/phd2/Documenti/embryo/embryo_to_send_gradcam/correct_blasto/ConvTran/D2015.11.23_S1389_I141_2/overlay_farneback"
input_dir = "/home/phd2/Documenti/embryo/embryo_to_send_gradcam/correct_no_blasto/ConvTran/D2015.01.15_S1202_I141_9/overlay_farneback"
save_folder = "/home/phd2/Documenti/embryo"
fps = 2  # Frames per second
# -------------------------------------------------

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [atoi(c) for c in re.split(r'(\d+)', text)]

def extract_timing(filename):
    """
    Extracts '1.0h' from '..._5_0_1.0h.jpg'
    Regex looks for: underscore + digits + dot + digits + 'h' + before file extension
    """
    match = re.search(r'_([0-9]+\.[0-9]+h)(?=\.[^\.]+$)', filename)
    if match:
        return match.group(1)
    return ""

def create_video_with_labels():
    # 1. Setup paths
    parent_name = os.path.basename(os.path.dirname(input_dir)) 
    video_name = f"{parent_name}_OF_{fps}fps.avi"
    save_path = os.path.join(save_folder, video_name)

    # 2. Get and sort images
    images = [img for img in os.listdir(input_dir) if img.lower().endswith((".png", ".jpg", ".jpeg", ".tif"))]
    images.sort(key=natural_keys)

    if not images:
        print(f"Error: No images found in {input_dir}")
        return

    # 3. Read first frame for dimensions
    first_frame_path = os.path.join(input_dir, images[0])
    frame0 = cv2.imread(first_frame_path)
    if frame0 is None:
        print(f"Error reading first frame: {first_frame_path}")
        return
        
    height, width, _ = frame0.shape

    # 4. Initialize VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'MJPG') 
    video = cv2.VideoWriter(save_path, fourcc, fps, (width, height))

    print(f"Processing {len(images)} frames with labels...")
    
    # 5. Process each frame
    for image_name in images:
        full_path = os.path.join(input_dir, image_name)
        frame = cv2.imread(full_path)
        
        if frame is not None:
            # --- LABELING LOGIC ---
            text = extract_timing(image_name)
            
            if text:
                # Font settings
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 0.5
                thickness = 1
                margin = 20
                
                # Calculate text size to position it
                (text_w, text_h), baseline = cv2.getTextSize(text, font, font_scale, thickness)
                
                # Coordinates for Bottom-Right
                x = width - text_w - margin
                y = height - margin
                
                # Draw Stroke (Black outline)
                cv2.putText(frame, text, (x, y), font, font_scale, (0, 0, 0), thickness + 3, cv2.LINE_AA)
                # Draw Text (White foreground)
                cv2.putText(frame, text, (x, y), font, font_scale, (255, 255, 255), thickness, cv2.LINE_AA)
            # ----------------------

            video.write(frame)

    video.release()
    # cv2.destroyAllWindows() # Removed to avoid Ubuntu/headless error
    
    print(f"Video saved to: {save_path}")

create_video_with_labels()

Processing 564 frames with labels...
Video saved to: /home/phd2/Documenti/embryo/D2015.01.15_S1202_I141_9_OF_2fps.avi
